In [9]:
! wget -O cell_classes.xlsx https://docs.google.com/spreadsheets/d/e/2PACX-1vTanLtzxD6OXpu3Ze4aNITlIMZEqfK3qrrcNiwFE6kA-YVnuzULp3dG3oYIe5gYAVj28QWZnGwzN_H6/pub\?output\=xlsx 

--2021-07-08 08:12:54--  https://docs.google.com/spreadsheets/d/e/2PACX-1vTanLtzxD6OXpu3Ze4aNITlIMZEqfK3qrrcNiwFE6kA-YVnuzULp3dG3oYIe5gYAVj28QWZnGwzN_H6/pub?output=xlsx
Resolving docs.google.com (docs.google.com)... 142.250.79.46, 2800:3f0:4004:80f::200e
Connecting to docs.google.com (docs.google.com)|142.250.79.46|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-08-2g-sheets.googleusercontent.com/pub/70cmver1f290kjsnpar5ku2h9g/iunem9rg6knvqakl3jvdd963og/1625742770000/111470779373223342477/*/e@2PACX-1vTanLtzxD6OXpu3Ze4aNITlIMZEqfK3qrrcNiwFE6kA-YVnuzULp3dG3oYIe5gYAVj28QWZnGwzN_H6?output=xlsx [following]
--2021-07-08 08:12:54--  https://doc-08-2g-sheets.googleusercontent.com/pub/70cmver1f290kjsnpar5ku2h9g/iunem9rg6knvqakl3jvdd963og/1625742770000/111470779373223342477/*/e@2PACX-1vTanLtzxD6OXpu3Ze4aNITlIMZEqfK3qrrcNiwFE6kA-YVnuzULp3dG3oYIe5gYAVj28QWZnGwzN_H6?output=xlsx
Resolving doc-08-2g-sheets.googleusercontent.com (doc-08-2g-sheets

In [10]:
import pandas as pd
cells = pd.read_excel("cell_classes.xlsx", sheet_name="cell classes")

In [11]:
cells.head()

,label,subclass of,stated in,described by source,aliases,anatomical location,develops from,instance of
0,human OPN1SW-positive S-cones,Q147298,\nQ92766837,NaN,S-cones,NaN,NaN,NaN
1,human L/M cones,Q147298,\nQ92766837,NaN,NaN,NaN,NaN,NaN
2,human OPN1MW-positive M-cones,human L/M cones,Q92766837,NaN,NaN,NaN,NaN,NaN
3,human OPN1LW-positive L-cones,human L/M cones,Q92766837,NaN,NaN,NaN,NaN,NaN
4,IgG B cell,Q188930,Q92826478,NaN,NaN,NaN,NaN,NaN


In [4]:
%run dicts.py

In [175]:
import re
import pickle

with open('celltypes_dict.pickle', 'rb') as handle:
    subclass_dict = pickle.load(handle)
    
print(subclass_dict)

subclass_dict["B cell"] =  'Q188930'

{'progenitor cell': 'Q1896218', 'thymocyte': 'Q7799635', 'mouse early thymic progenitor cell': 'Q106303241', 'mouse double-negative (DN) cell': 'Q106303242', 'mouse  DN1 cells (CD44+CD252)': 'Q106303259', 'mouse DN2 cells (CD44+CD25+)': 'Q106303260', 'club cell': 'Q645712', 'fibroblast': 'Q463418', 'glycinergic neuron': 'Q101062691', 'excitatory neuron': 'Q104519429', 'cell': 'Q7868', 'cytotrophoblasts': 'Q105321205', 'endothelial cell': 'Q11394395', 'macrophage': 'Q184204', 'fetal liver island macrophage': 'Q106410670', 'erythroblast': 'Q3296909', 'Human fetal liver island macrophage': 'Q106410701', 'pigment cell': 'Q504699', 'neural crest cell': 'Q66592575', 'epithelial cell': 'Q15176415', 'coronary artery SMC': 'Q107297649', 'human coronary artery SMC': 'Q107297653', 'fibromyocyte': 'Q107297654', 'dendritic cell': 'Q506253', 'human dendritic cell': 'Q101404869', 'human CD1C+ cDC': 'Q107297680', 'human CD11C+ conventional dendrintic cells': 'Q107297679', 'human CD1C+ A cDC': 'Q107297

In [5]:
from wikidataintegrator import wdi_login
import getpass

pwd = getpass.getpass()
login_instance = wdi_login.WDLogin(user='TiagoLubiana', pwd=pwd)

 ············


In [7]:
from wikidataintegrator import wdi_core
import time

for index, row in cells.iterrows():

    data_for_item = []     
    stated_in_full_reference = wdi_core.WDItemID(value=row["stated in"], prop_nr="P248", is_reference=True)
    references = [[stated_in_full_reference]]
    

    label = row["label"]
    print(f"Running code for {label} ")

    if "human" in label:
      description = "cell type in Homo sapiens"
      taxon = "Q15978631"
        
      data_for_item.append( 
        wdi_core.WDItemID(value=taxon,
                        prop_nr="P703",
                         references=references)
    )  
      
    elif "mouse" in label:
      description = "cell type in Mus musculus"
      data_for_item.append( 
        wdi_core.WDItemID(value=taxon,
                        prop_nr="P703",
                         references=references)
    )  

    elif "zebrafish" in label:
      description = "cell type in Danio rerio"
      taxon = "Q169444"
      data_for_item.append( 
        wdi_core.WDItemID(value=taxon,
                        prop_nr="P703",
                         references=references)
    )  
    else:
      description = "cell type"

    try:
        if  re.findall("Q[0-9]*", row["subclass of"]):
            subclass = row["subclass of"]
        else:
            subclass = subclass_dict[row["subclass of"]]            
        
        data_for_item.append( 
        wdi_core.WDItemID(value=subclass,
                        prop_nr="P279")
    )  

        if str(row["anatomical location"]) != "nan":
            if  re.findall("Q[0-9]*", row["anatomical location"]):
                part_of = row["anatomical location"]
            else:
                part_of = part_of_dict[row["anatomical location"]]

            data_for_item.append( 
            wdi_core.WDItemID(value=part_of,
                            prop_nr="P927",
                             references=references)
        )          

        if str(row["described by source"]) != "nan":
            data_for_item.append( 
            wdi_core.WDItemID(value=row["described by source"],
                            prop_nr="P1343")
                        )
        
       
    except:
        print("")
        print("failed: ")
        print(row)
        break
    
    if str(row["aliases"]) != "nan":
        aliases = row["aliases"].split("|")
    else:
        aliases = []

    if str(row["instance of"]) != "nan":
      instance = instance_dict[row["instance of"]]
    else: 
      instance = instance_dict["cell type"]
    print(instance)
    data_for_item.append( 
    wdi_core.WDItemID(value=instance,
                    prop_nr="P31",
                    references=references)
    )
    wd_item = wdi_core.WDItemEngine(data=data_for_item)
    wd_item.set_label(label=label, lang="en")
    wd_item.set_aliases(aliases, lang="en")
    wd_item.set_description(description, lang="en")
    try:
        wd_item.write(login_instance)
        subclass_dict[label] = wd_item.wd_item_id
    except:
        print("Something went wrong")
    
    time.sleep(1)

        

    

ValueError: Invalid WD item ID, format must be "Q[0-9]*"

In [177]:
with open('celltypes_dict.pickle', 'wb') as handle:
    pickle.dump(subclass_dict, handle, protocol=pickle.HIGHEST_PRO
                OCOL)


SyntaxError: invalid syntax (<ipython-input-177-d0015768fd33>, line 3)